<a href="https://colab.research.google.com/github/anatorres09/DE-MRI_Classificacio_CNN/blob/main/preprocessat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%tensorflow_version 2.x
# Clonar el repositori del GitHUb que conté les imatges
!git clone https://github.com/anatorres09/DE-MRI_Classificacio_CNN.git
! ls

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.
fatal: destination path 'DE-MRI_Classificacio_CNN' already exists and is not an empty directory.
DE-MRI_Classificacio_CNN  sample_data


In [3]:
# Connectar a Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
#Importar llibreries
import os
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from sklearn.model_selection import train_test_split

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

from mpl_toolkits.axes_grid1 import ImageGrid
from matplotlib.gridspec import GridSpec

from skimage.filters import median, rank
from skimage.exposure import equalize_adapthist

from scipy import ndimage
from tensorflow import keras
from tensorflow.keras import layers
import nibabel as nib
import os
import urllib.request

import random
import cv2
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

from glob import glob

import shutil
from sklearn.preprocessing import LabelEncoder


In [5]:
#ELIMINAR CARPETA CORR_ENTRENAMENT (evitar duplicacions imatges 2D)
# Directori on es troben les imatges en format NifTI
data_path = '/content/DE-MRI_Classificacio_CNN/dataset' 
drive_folder = '/content/drive/MyDrive/CNN'

# Eliminar la carpeta corr_entrenament si ja existeix

# Verificar la existencia de la carpeta corr_entrenament

def eliminar_contingut_carpeta(carpeta):
  if os.path.exists(directori_corr_entrenament):
    for arxiu in os.listdir(carpeta):
        ruta_arxiu = os.path.join(carpeta, arxiu)
        if os.path.isfile(ruta_arxiu):
            os.remove(ruta_arxiu)
        elif os.path.isdir(ruta_arxiu):
            shutil.rmtree(ruta_arxiu)
  else: print("La carpeta corr_entrenament no existeix. No es realitza cap acció.")


#directori_corr_entrenament = os.path.join(data_path, "corr_entrenament")
directori_corr_entrenament = os.path.join(drive_folder, "corr_entrenament")


eliminar_contingut_carpeta(directori_corr_entrenament)



La carpeta corr_entrenament no existeix. No es realitza cap acció.


In [7]:
#CONVERTIR IMATGES 3D A 2D
def convertir_imatge_3D_a_2D(nom_arxiu, carpeta_sortida):
    # Carregar l'arxiu NIfTI
    imatge_nifti = nib.load(nom_arxiu)
    dades = imatge_nifti.get_fdata()

    # Obtenir les dimensions de la imatge
    shape = dades.shape

    # Crear la carpeta de sortida per la imatge si no existeix
    if not os.path.exists(carpeta_sortida):
        os.makedirs(carpeta_sortida)

    # Realitzar els talls i guardar les imatges 2D resultants
    for i in range(shape[2]):
        # Obtenir el tall en l'índex corresponent
        tall = dades[:, :, i]

        # Crear una figura i mostrar el tall
        fig = plt.figure()
        plt.imshow(tall, cmap='gray')
        plt.axis('off')

        # Guardar la imatge en la carpeta de sortida
        nom_sortida = os.path.join(carpeta_sortida, f'tall_{i}.png')
        plt.savefig(nom_sortida)
        plt.close(fig)

    print(f"S'han generat {shape[2]} imatges 2D en talls per {nom_arxiu}.")

# Carpeta d'entrada per imatges normals
carpeta_entrada_normal = os.path.join(data_path, "entrenament", "normal")

# Carpeta d'entrada per imatges patològiques
carpeta_entrada_patologic = os.path.join(data_path, "entrenament", "patologic")

# Carpeta d'entrada per imatges patològiques
carpeta_entrada_test = os.path.join(data_path, "test")

# Carpeta de sortida per les imatges 2D convertides (normals)
#carpeta_sortida_normal = os.path.join(data_path, "corr_entrenament", "corr_normal")
carpeta_sortida_normal = os.path.join(drive_folder, "corr_entrenament", "corr_normal")

# Carpeta de sortida per les imatges 2D convertides (patològiques)
#carpeta_sortida_patologic = os.path.join(data_path, "corr_entrenament", "corr_patologic")
carpeta_sortida_patologic = os.path.join(drive_folder, "corr_entrenament", "corr_patologic")

# Carpeta de sortida per les imatges 2D convertides (test)
carpeta_sortida_test = os.path.join(drive_folder, "test")

# Processar imatges normals
arxius_normal = os.listdir(carpeta_entrada_normal)
for arxiu in arxius_normal:
    if arxiu.endswith('.nii.gz') or arxiu.endswith('.nii'):
        ruta_arxiu = os.path.join(carpeta_entrada_normal, arxiu)
        carpeta_sortida_imatge = os.path.join(carpeta_sortida_normal, os.path.splitext(arxiu)[0])
        convertir_imatge_3D_a_2D(ruta_arxiu, carpeta_sortida_imatge)

# Processar imatges patològiques
arxius_patologic = os.listdir(carpeta_entrada_patologic)
for arxiu in arxius_patologic:
    if arxiu.endswith('.nii.gz') or arxiu.endswith('.nii'):
        ruta_arxiu = os.path.join(carpeta_entrada_patologic, arxiu)
        carpeta_sortida_imatge = os.path.join(carpeta_sortida_patologic, os.path.splitext(arxiu)[0])
        convertir_imatge_3D_a_2D(ruta_arxiu, carpeta_sortida_imatge)


# Processar imatges test
arxius_test = os.listdir(carpeta_entrada_test)
for arxiu in arxius_test:
    if arxiu.endswith('.nii.gz') or arxiu.endswith('.nii'):
        ruta_arxiu = os.path.join(carpeta_entrada_test, arxiu)
        carpeta_sortida_imatge = os.path.join(carpeta_sortida_test, os.path.splitext(arxiu)[0])
        convertir_imatge_3D_a_2D(ruta_arxiu, carpeta_sortida_imatge)

S'han generat 7 imatges 2D en talls per /content/DE-MRI_Classificacio_CNN/dataset/entrenament/normal/Case_N020.nii.
S'han generat 8 imatges 2D en talls per /content/DE-MRI_Classificacio_CNN/dataset/entrenament/normal/Case_N024.nii.
S'han generat 8 imatges 2D en talls per /content/DE-MRI_Classificacio_CNN/dataset/entrenament/normal/Case_N046.nii.
S'han generat 7 imatges 2D en talls per /content/DE-MRI_Classificacio_CNN/dataset/entrenament/normal/Case_N075.nii.
S'han generat 7 imatges 2D en talls per /content/DE-MRI_Classificacio_CNN/dataset/entrenament/normal/Case_N058.nii.
S'han generat 6 imatges 2D en talls per /content/DE-MRI_Classificacio_CNN/dataset/entrenament/normal/Case_N084.nii.
S'han generat 7 imatges 2D en talls per /content/DE-MRI_Classificacio_CNN/dataset/entrenament/normal/Case_N030.nii.
S'han generat 8 imatges 2D en talls per /content/DE-MRI_Classificacio_CNN/dataset/entrenament/normal/Case_N049.nii.
S'han generat 7 imatges 2D en talls per /content/DE-MRI_Classificacio_CN